### Deep Nets with TF Abstractions
Let's explore a few of the various abstractions that TensorFlow offers. You can check out the tf.contrib documentation for more options.

### The Data

To compare these various abstractions we'll use a dataset easily available from the SciKit Learn library. The data is comprised of the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators. There are thirteen different
measurements taken for different constituents found in the three types of wine. We will use the various TF Abstractions to classify the wine to one of the 3 possible labels.

First let's show you how to get the data:

In [4]:
from sklearn.datasets import load_wine

In [5]:
wine_data = load_wine()

In [6]:
type(wine_data)

sklearn.utils.Bunch

In [7]:
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [8]:
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [9]:
feature_data = wine_data['data']

In [10]:
labels = wine_data['target']

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(feature_data, 
                                                    labels, 
                                                    test_size=0.33, 
                                                    random_state=101)

### Scale the Data

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
scaler = MinMaxScaler()

In [15]:
scaled_x_train = scaler.fit_transform(X_train)

In [16]:
scaled_x_test = scaler.transform(X_test)

### Abstractions

In [14]:
import tensorflow as tf

In [15]:
from tensorflow import estimator

In [16]:
X_train.shape

(119, 13)

In [17]:
feat_col = [tf.feature_column.numeric_column("x", shape=[13])]

In [18]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13], 
                                     feature_columns=feat_col, 
                                     n_classes=3,
                                     optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmprxv4ydhf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12ee8aef0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},
                                           y=y_train, 
                                           shuffle=True, 
                                           batch_size=10, 
                                           num_epochs=5)

In [20]:
deep_model.train(input_fn=input_fn, steps=500)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmprxv4ydhf/model.ckpt.
INFO:tensorflow:loss = 11.126078, step = 1
INFO:tensorflow:Saving checkpoints for 60 into /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmprxv4ydhf/model.ckpt.
INFO:tensorflow:Loss for final step: 5.1188183.


In [21]:
input_fun_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test}, shuffle=False)

In [22]:
preds = list(deep_model.predict(input_fun_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/folders/j8/4vht5fhd3rg9wv8rx18tt20c0000gp/T/tmprxv4ydhf/model.ckpt-60
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [23]:
predictions = [p['class_ids'][0] for p in preds]

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

In [25]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.53      1.00      0.69        20
           1       0.76      0.64      0.70        25
           2       0.00      0.00      0.00        14

    accuracy                           0.61        59
   macro avg       0.43      0.55      0.46        59
weighted avg       0.50      0.61      0.53        59



/Users/Slimn/anaconda3/envs/tfdl/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---
---

## TensorFlow Keras

In [26]:
import tensorflow as tf

In [27]:
from tensorflow.contrib.keras import models

In [28]:
dnn_keras_model = models.Sequential()

In [29]:
from tensorflow.contrib.keras import layers

In [30]:
dnn_keras_model.add(layers.Dense(units=13, input_dim=13,activation='relu'))

In [31]:
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))
dnn_keras_model.add(layers.Dense(units=13, activation='relu'))

In [32]:
dnn_keras_model.add(layers.Dense(units=3, activation='softmax'))

In [33]:
from tensorflow.contrib.keras import losses, optimizers, activations

In [34]:
dnn_keras_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train Model

In [35]:
dnn_keras_model.fit(scaled_x_train, y_train, epochs=50)

Epoch 1/50
119/119 [==============================] - 2s 20ms/sample - loss: 1.1059 - acc: 0.2521
Epoch 2/50
119/119 [==============================] - 0s 609us/sample - loss: 1.0947 - acc: 0.3109
Epoch 3/50
119/119 [==============================] - 0s 432us/sample - loss: 1.0844 - acc: 0.3529
Epoch 4/50
119/119 [==============================] - 0s 522us/sample - loss: 1.0739 - acc: 0.3445
Epoch 5/50
119/119 [==============================] - 0s 1ms/sample - loss: 1.0627 - acc: 0.4958
Epoch 6/50
119/119 [==============================] - 0s 764us/sample - loss: 1.0523 - acc: 0.5882
Epoch 7/50
119/119 [==============================] - 0s 977us/sample - loss: 1.0425 - acc: 0.6303
Epoch 8/50
119/119 [==============================] - 0s 280us/sample - loss: 1.0332 - acc: 0.6303
Epoch 9/50
119/119 [==============================] - 0s 306us/sample - loss: 1.0237 - acc: 0.6303
Epoch 10/50
119/119 [==============================] - 0s 834us/sample - loss: 1.0132 - acc: 0.6303s - loss: 1.0

In [36]:
predictions=dnn_keras_model.predict_classes(scaled_x_test)

In [37]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90        20
           1       0.88      0.92      0.90        24
           2       1.00      0.93      0.97        15

    accuracy                           0.92        59
   macro avg       0.93      0.92      0.92        59
weighted avg       0.92      0.92      0.92        59



## Layers API
https://www.tensorflow.org/tutorials/layers

In [32]:
import tensorflow as tf
import pandas as pd
tf.reset_default_graph()

In [33]:
onehot_y_train = pd.get_dummies(y_train).as_matrix()

/Users/Slimn/anaconda3/envs/tfdl/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


### Parameters

In [34]:
num_feat = 13
num_hidden1 = 13
num_hidden2 = 13
num_outputs = 3

In [35]:
learning_rate = 0.01

### Placeholder

In [36]:
from tensorflow.contrib.layers import fully_connected

In [37]:
X = tf.placeholder(tf.float32,shape=[None, num_feat])
y_true = tf.placeholder(tf.float32,shape=[None, 3])
actf = tf.nn.relu

### Create Layers

In [38]:
hidden1 = fully_connected(X, num_hidden1, activation_fn=actf)

In [39]:
hidden2 = fully_connected(hidden1, num_hidden2, activation_fn=actf)

In [40]:
output = fully_connected(hidden2, num_outputs)

### Loss Function

In [41]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

### Optimizer

In [42]:
optimizer = tf.train.AdamOptimizer(learning_rate)

In [43]:
train = optimizer.minimize(loss)

### Init

In [44]:
init = tf.global_variables_initializer()

In [46]:
training_steps = 1000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(train,feed_dict={X:scaled_x_train,y_true:onehot_y_train})
        
    # Get Predictions
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    preds = tf.argmax(logits,axis=1)
    
    results = preds.eval()

In [47]:
from sklearn.metrics import classification_report

In [48]:
print(classification_report(results,y_test))

              precision    recall  f1-score   support

           0       1.00      0.61      0.75        33
           1       1.00      0.96      0.98        26
           2       0.00      0.00      0.00         0

    accuracy                           0.76        59
   macro avg       0.67      0.52      0.58        59
weighted avg       1.00      0.76      0.85        59



/Users/Slimn/anaconda3/envs/tfdl/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
